<a href="https://www.kaggle.com/code/dewasahu/pdf-extractor?scriptVersionId=143433409" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#install all packages
!pip install tokenizer transformers scikit-learn pypdf==3.16.0 nltk pandarallel pandas==2.1.0 datasets #pyspark pyarrow

In [ ]:
#all imports
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import datasets
import transformers

In [ ]:
#resume_data=pd.read_csv("kaggle resume dataset filepath") change it accordingly
resume_data=pd.read_csv("/kaggle/input/resume-dataset/Resume/Resume.csv")
resume_data=resume_data.drop(["Resume_html"],axis=1)

In [ ]:
#getting all text from a pdf
from pypdf import PdfReader

def pdf_text(filePath:str)->str:
    reader = PdfReader(filePath)
    text=""
    for page in reader.pages:
        text+=page.extract_text()
    return text

In [ ]:
#doing text cleaninig and tokenization
from nltk import pos_tag
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
import string
import re

puncuation=set(string.punctuation)
stop_words_english=set(stopwords.words("english"))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    sentences = sent_tokenize(text)
    features = {'feature': ""}

    for sent in sentences:
        for criteria in ['skills', 'education']:
            if criteria in sent:
                words = word_tokenize(sent)
                words = [word for word in words if word not in stop_words_english]
                # POS tagger to identify and remove stop words and other irrelevant words
                tagged_words = pos_tag(words)
                filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
                features['feature'] += " ".join(filtered_words)

    return features

In [ ]:
progress_bar=tqdm(range(len(resume_data)))
# extract text from csv and get the text from pdf
def process(df):
    #uncomment when using pdf_text function .. done to save time
    id=df['ID']
    category=df['Category'] 
    text=pdf_text(f"/kaggle/input/resume-dataset/data/data/{category}/{id}.pdf")
   # text=str(df["Resume_str"])
    features=preprocess_text(text)
    df['Feature']=features['feature']
    progress_bar.update(1)
    return df
      
#applying processing to resume_data
resume_data=resume_data.apply(process,axis=1)
resume_data=resume_data.drop(columns=['Resume_str'])



In [ ]:
#to avoid computing again and save time
#resume_data.to_csv("/path_to_save_processed_resume_data")
resume_data.to_csv("/kaggle/working/resume_data.csv",index=False)

In [ ]:
#resume_data=pd.read_csv("/path_to_save_processed_resume_data")
resume_data=pd.read_csv("/kaggle/working/resume_data.csv")

In [ ]:
resume_data.head()

In [ ]:
resume_data['Category'].value_counts().sort_index().plot(kind="bar",figsize=(12,6))

In [ ]:
#getting job description
num_desc=15 #number of jod_description to fetch from job_description_data

#job_description=pd.read_csv("job_description_data.csv")
job_description=pd.read_csv("/kaggle/input/resume-and-job-description/training_data.csv")

#choosing 15 job description
job_description=job_description[["job_description","position_title"]][:num_desc]

In [ ]:
# processing the text of the job description
job_description['Features']=job_description['job_description'].apply(lambda x : preprocess_text(x)['feature'])


In [ ]:
#choosing 15 job description
job_description=job_description[["job_description","position_title"]][:num_desc]
# processing the text of the job description
job_description['Features']=job_description['job_description'].apply(lambda x : preprocess_text(x)['feature'])


In [ ]:
job_description.head()

In [ ]:
#using pytorch along with 🤗transformers...
from transformers import AutoModel, AutoTokenizer
import torch

device="cuda"if torch.cuda.is_available() else "cpu"

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#putting model and tokenizer to gpu for faster processing 
model.to(device)


def get_embeddings(text):
    inputs = tokenizer(str(text), return_tensors="pt",truncation=True,padding=True).to(device)
    outputs = model(**inputs)
    #getting the embedding to cpu
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().to("cpu").numpy() 
    return embeddings


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd


# Calculate embeddings for all job descriptions and resumes
job_desc_embeddings = np.array([get_embeddings(desc) for desc in job_description['Features']])
resume_embeddings = np.array([get_embeddings(text) for text in resume_data['Feature']])



In [ ]:
#sqeezing job embeding 
job_desc_embeddings=job_desc_embeddings.squeeze()
resume_embeddings=resume_embeddings.squeeze()
resume_embeddings.shape,job_desc_embeddings.shape

In [ ]:
# Initialize a DataFrame to store the results
result_df = pd.DataFrame(columns=['jobId', 'resumeId', 'similarity', 'domainResume', 'domainDesc'])

# top k-resumes
k=5


In [ ]:

# Iterate over job descriptions
for i, job_desc_emb in enumerate(job_desc_embeddings):
    job_desc_id = i
    job_title = job_description['position_title'].iloc[i]

    # Compute cosine similarities between the current job description and all resumes
    similarities = cosine_similarity([job_desc_emb], resume_embeddings )

    # Get the indices of the top-k most similar resumes
    top_k_indices = np.argsort(similarities[0])[::-1][:k]
   
    # Extract the relevant information and add it to the result DataFrame
    for j in top_k_indices:
        resume_id = resume_data['ID'].iloc[j]
        work_domain = resume_data['Category'].iloc[j]
        similarity_score = similarities[0][j]
        
        result_df.loc[i+j] = [job_desc_id, resume_id, similarity_score, work_domain,job_title ]
        

# Sort the results by similarity score (descending)
result_df = result_df.sort_values(by='similarity', ascending=False)



In [ ]:
result_df.head()

In [ ]:
result_group=result_df.groupby("jobId")
result_group

In [ ]:
#example of group data
result_group.get_group(0)

In [ ]:
for i in range(num_desc):
    print()
    print("jobId---cosineSimilarity---domainResume---domainDesc")
    print(result_group.get_group(i).values[0])
    print()